In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/barbara/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def get_unique_words(text):
    words = [word for message in text for word in message]
    return set(words)

def co_occurence_matrix_with_window(text, window_size, stop_words):
    '''
    Calculates the co-occurence matrix using a sliding window of size window_size as context    
    Output is (unique) words x (unique) words
    '''
    unique_words = get_unique_words(text)
    n = len(unique_words) #number of unique words
    co_matrix = pd.DataFrame(data=np.zeros([n,n]), columns=unique_words, index=unique_words, dtype='int')

    #for each word in each message, counts +1 for each pair [word,context]
    for message in text:
        msg_len = len(message)
        for i, word in enumerate(message):
            first = max(i - window_size, 0)
            last = min(i + window_size + 1, msg_len)
            for context in message[first:last]:
                co_matrix.loc[word,context] += 1
    return co_matrix  
    
def word_occurence_matrix(text, target=None, stop_words=None, binary=True, preprocess_text=False):  
    '''
    Output is messages x (unique) words
    
    If binary=True, then each element represents if the word is in the message or not.
    Otherwise, it represents the count of how many times that word appears in that message.
    ''' 
    if target:
        text = list(filter(lambda x : target in x, text)) #Filter comments in which target word is present
        
    preprocessor = CountVectorizer(strip_accents='unicode').build_preprocessor()   
    if stop_words:        
        stop_words = [preprocessor(word) for word in stop_words] #preprocesses stop words
    if preprocess_text:
        text = [preprocessor(msg) for msg in text] #preprocesses text
        
    #calculates word count for each message
    vectorizer = CountVectorizer(strip_accents='unicode', stop_words=stop_words, binary=binary)
    X = vectorizer.fit_transform(text).toarray()
    
    labels = vectorizer.get_feature_names()
    
    return X, labels

def co_occurence_matrix(word_occurence_matrix):
    '''
    Co-occurence matrix created based on the word count/occurence matrix.     
    In other words, this co-occurence matrix will consider each message as the context for all words in it.
    '''
    co_X = word_occurence_matrix.T @ word_occurence_matrix
    np.fill_diagonal(co_X, 0)
    return co_X

def score_ngrams(text, freq_filter=1, score_metric=BigramAssocMeasures().pmi):
    '''
    Score n-grams using score_metric. Defaults to PMI.
    '''
    words = [word for msg in text for word in msg.split()]
    finder = BigramCollocationFinder.from_words(words)
    finder.apply_freq_filter(freq_filter)
    return finder.score_ngrams(score_metric)

### Dados

In [4]:
comments = pd.read_csv('../../dados/youtube/comentarios_sorted_votes.csv')
stop_words = [word.rstrip() for word in open('../stopwords.txt')]

#Preprocesses text
preprocessor = CountVectorizer(strip_accents='unicode').build_preprocessor() #lowercase and strip accents
stop_words = [preprocessor(word) for word in stop_words]
comments['text'] = [preprocessor(msg) for msg in comments['text']]
comments['text'] = [' '.join([word for word in RegexpTokenizer(r'\w+').tokenize(msg) if not word in stop_words])
                    for msg in comments['text']]

text = comments['text'].dropna()

N = 30 #Consider only the N most frequent words

In [9]:
comments_by_channel = []
channels = []
for channel, group in comments.groupby('uploader'):
    channel_comments = group['text']
    comments_by_channel.append(channel_comments)
    channels.append(channel)

### 1. Co-ocorrência 

#### 1.1. Co-cocorrência com uma palavra pré-definida

In [6]:
target = 'cloroquina'

X, labels = word_occurence_matrix(text, target=target, stop_words=stop_words)

word_count = X.sum(axis=0) #no. of comments containing A in which each word appears
word_count = sorted(zip(labels, word_count), reverse=True, key=lambda tupl : tupl[1])
print(f'{N} palavras que mais ocorrem nos mesmos comentários em que "{target}" ocorre:\n')
print(word_count[:N])

30 palavras que mais ocorrem nos mesmos comentários em que "cloroquina" ocorre:

[('cloroquina', 615), ('hidroxicloroquina', 497), ('ivermectina', 360), ('covid', 209), ('azitromicina', 163), ('medicos', 154), ('tratamento', 153), ('dr', 151), ('sobre', 149), ('uso', 147), ('pessoas', 145), ('pra', 142), ('estudo', 138), ('19', 135), ('medico', 134), ('pode', 119), ('todos', 114), ('bem', 108), ('contra', 106), ('dra', 103), ('tomar', 103), ('agora', 102), ('protocolo', 99), ('https', 96), ('porque', 96), ('sim', 95), ('sintomas', 92), ('saude', 91), ('virus', 91), ('pois', 90)]


In [16]:
cc_by_channel = []
cc_channels = []
cc_empty = []
for i, channel in enumerate(comments_by_channel):
    try:
        X, labels = word_occurence_matrix(channel, target=target, stop_words=stop_words)
        word_count = X.sum(axis=0) #no. of comments containing A in which each word appears
        word_count = sorted(zip(labels, word_count), reverse=True, key=lambda tupl : tupl[1])
        cc_by_channel.append(word_count)
        cc_channels.append(channels[i])
    except: #target word doesn't occur in the channel
        cc_empty.append(channels[i])
    

df_cc = pd.DataFrame(cc_by_channel, index = cc_channels).T
print(f'Nenhuma ocorrência de "{target}" nos canais: ', cc_empty)
print(f'Palavras que mais co-ocorrem nos comentários com "{target}" em cada canal:')
df_cc.head(10)

Nenhuma ocorrência de "cloroquina" nos canais:  ['Dr. Fernando Gomes']
Palavras que mais co-ocorrem nos comentários com "cloroquina" em cada canal:


,Dr. Alain Dutra,Dr. Alvaro Galvão,Dr. Felipe Ades MD PhD,Dr. Lair Ribeiro Oficial,Drauzio Varella,Julio Pereira - Neurocirurgião,Lucy Kerr
0,"(cloroquina, 68)","(cloroquina, 31)","(cloroquina, 3)","(cloroquina, 20)","(cloroquina, 59)","(cloroquina, 44)","(cloroquina, 390)"
1,"(ivermectina, 66)","(ivermectina, 27)","(ainda, 2)","(hidroxicloroquina, 10)","(hidroxicloroquina, 35)","(azitromicina, 30)","(hidroxicloroquina, 348)"
2,"(hidroxicloroquina, 63)","(hidroxicloroquina, 16)","(deus, 2)","(lair, 9)","(drauzio, 18)","(hidroxicloroquina, 25)","(ivermectina, 242)"
3,"(azitromicina, 29)","(dr, 12)","(dexametasona, 2)","(tratamento, 9)","(covid, 17)","(ivermectina, 16)","(covid, 141)"
4,"(contra, 21)","(agora, 11)","(pra, 2)","(azitromicina, 8)","(dr, 16)","(ainda, 15)","(estudo, 124)"
5,"(agora, 20)","(covid, 11)","(cancer, 1)","(sim, 7)","(azitromicina, 14)","(covid, 15)","(sobre, 113)"
6,"(dr, 20)","(medicos, 11)","(conheco, 1)","(19, 6)","(medico, 14)","(tomar, 15)","(medicos, 107)"
7,"(covid, 19)","(vai, 10)","(continuo, 1)","(contra, 6)","(pra, 14)","(vai, 15)","(tratamento, 101)"
8,"(pessoas, 19)","(pessoas, 9)","(cotonete, 1)","(covid, 6)","(remedio, 14)","(deus, 14)","(uso, 98)"
9,"(todos, 19)","(pra, 9)","(curadas, 1)","(outras, 6)","(sobre, 13)","(19, 12)","(pessoas, 97)"


#### 1.2. N-gramas

Considerações:

- PMI - favorece n-gramas raros. Muitos n-gramas com a mesma score (a mais alta)

#### PMI

In [20]:
pmi = BigramAssocMeasures().pmi
print(f'{N} com score mais alta:')
score_ngrams(text, score_metric=pmi, freq_filter=10)[:N]

30 com score mais alta:


[(('assalim', 'massoni'), 16.49855785890024),
 (('katti', 'assalim'), 16.49855785890024),
 (('zmnwnb8', 'ajg'), 16.49855785890024),
 (('hay', 'sama'), 16.361054335150307),
 (('1faipqlsdafkdw955ygar_0em_ndfiqcinqtyxyyliv689ylj9dfcxia', 'viewform'),
  16.361054335150303),
 (('27176561', 'drogao'), 16.361054335150303),
 (('deby', 'freiitas'), 16.361054335150303),
 (('dizzy', 'roses'), 16.361054335150303),
 (('ted', 'posseidon'), 16.361054335150303),
 (('freddy', 'yanez'), 16.23552345306645),
 (('gilselaine', 'pavan'), 16.23552345306645),
 (('hipnosisya', 'freddy'), 16.23552345306645),
 (('pavan', 'brioli'), 16.23552345306645),
 (('arn', 'sussekind'), 16.235523453066445),
 (('five', 'nights'), 16.235523453066445),
 (('reuber', 'felicio'), 16.235523453066445),
 (('park', 'jimin'), 16.120046235646512),
 (('alegrar', 'diaas'), 16.12004623564651),
 (('grow', 'rich'), 16.109992570982588),
 (('rivianne', 'ribeirohhftifi'), 16.013131031729998),
 (('gordiola', 'producoes'), 15.913595358179082),
 (

In [21]:
pmi_by_channel = []
for i, channel in enumerate(comments_by_channel):
    pmi_scores = score_ngrams(channel, score_metric=pmi, freq_filter=10)
    pmi_by_channel.append(pmi_scores)

df_cc = pd.DataFrame(pmi_by_channel, index = channels).T
df_cc.head(10)

,Dr. Alain Dutra,Dr. Alvaro Galvão,Dr. Felipe Ades MD PhD,Dr. Fernando Gomes,Dr. Lair Ribeiro Oficial,Drauzio Varella,Julio Pereira - Neurocirurgião,Lucy Kerr
0,"((abraao, guto), 13.42655305520809)","((youtu, be), 11.068874224185755)","((falta, ar), 7.637727962710633)","((deus, abencoe), 8.005624549193879)","((aquecimento, global), 10.799966499313511)","((https, bit), 15.021040686333661)","((hay, sama), 14.743514336924706)","((cel, celu), 13.825932668291175)"
1,"((assalim, massoni), 13.42655305520809)","((continente, africano), 10.80583981835196)","((boa, noite), 7.518739926493543)","((very, informative), 6.519537298555528)","((arimathea, gomes), 10.799966499313511)","((rivianne, ribeirohhftifi), 15.021040686333661)","((reuber, felicio), 14.743514336924706)","((27176561, drogao), 13.688429144541239)"
2,"((katti, assalim), 13.42655305520809)","((fake, news), 10.720950920765446)","((deus, abencoe), 6.583834954715432)","((good, work), 6.511477211658388)","((sars, cov), 10.39349101353809)","((dizzy, roses), 14.883537162583725)","((roger, delgado), 14.743514336924706)","((arn, sussekind), 13.56289826245738)"
3,"((noemi, firmo), 13.289049531458156)","((comprovacao, cientifica), 10.18233227396904)","((gracas, deus), 6.58383495471543)","((gomes, pinto), 6.274647024867269)","((cidade, wuhan), 10.35250752234229)","((grow, rich), 14.758006280499867)",((1faipqlsdafkdw955ygar_0em_ndfiqcinqtyxyyliv6...,"((freddy, yanez), 13.56289826245738)"
4,"((edielson, nogueira), 13.048041431954362)","((ultimas, noticias), 10.018248151115783)","((dor, cabeca), 5.876015706208742)","((gomez, pinto), 6.116101085555448)","((watch, v), 10.259398117950807)","((amp, 76), 14.53561385916342)","((deby, freiitas), 14.60601081317477)","((gilselaine, pavan), 13.56289826245738)"
5,"((isaac, nt), 12.941126228037849)","((efeitos, colaterais), 9.951416642637577)","((covid, 19), 5.815272515654884)","((boa, noite), 6.039000686398921)","((oleo, coco), 10.252478704011018)","((free, fire), 14.53561385916342)","((ted, posseidon), 14.60601081317477)","((hipnosisya, freddy), 13.56289826245738)"
6,"((renato, trovao), 12.578556148653142)","((uol, br), 9.80583981835196)","((deu, negativo), 5.616666347182804)","((boa, tarde), 5.910467316153536)","((youtu, be), 10.17847812256724)","((jack, taylor), 14.53561385916342)","((five, nights), 14.480479931090912)","((pavan, brioli), 13.56289826245738)"
7,"((adriana, cz), 12.578556148653139)","((121, 150), 9.788766304993018)","((muita, dor), 5.599175500849915)","((thank, you), 5.903196736269681)","((youtube, watch), 9.412943376204263)","((gordiola, producoes), 14.436078185612507)","((usp, sf_link), 14.480479931090912)","((drogao, net), 13.447421045037446)"
8,"((bonanne, vinte), 12.578556148653139)","((2020, 07), 9.780304726244823)","((deu, positivo), 5.252918076600814)","((i, love), 5.579594436281495)","((cov, 2), 9.108088794675842)","((bit, ly), 14.342968781221025)","((viewform, usp), 14.480479931090912)","((cgb, tdttqxq), 13.447421045037444)"
9,"((kkkkkkkkkkk, kkkkkkkkkkk), 12.519662459599571)","((91, 120), 9.744372185634564)","((14, dias), 5.160153361027334)","((you, are), 4.9645972809338215)","((https, youtu), 8.945817365776966)","((marcelo51, m51), 14.342968781221025)","((park, jimin), 14.365002713670977)","((neurociencia, felicidade), 13.447421045037444)"


#### likelihood ratio

In [22]:
lr = BigramAssocMeasures().likelihood_ratio
print(f'{N} com score mais alta:')
score_ngrams(text, score_metric=lr, freq_filter=10)[:N]

30 com score mais alta:


[(('covid', '19'), 16892.947470233976),
 (('falta', 'ar'), 16186.704051448698),
 (('olfato', 'paladar'), 14115.073150518603),
 (('gracas', 'deus'), 12369.146510580387),
 (('dor', 'cabeca'), 11614.437174829865),
 (('deus', 'abencoe'), 11288.143898218037),
 (('youtu', 'be'), 11039.053082917235),
 (('dra', 'lucy'), 10155.46757782443),
 (('dr', 'drauzio'), 10016.641748516702),
 (('https', 'youtu'), 8958.369513217573),
 (('sentir', 'cheiro'), 8539.500821771626),
 (('15', 'dias'), 8298.801986814038),
 (('https', 'www'), 8007.512202706735),
 (('lucy', 'kerr'), 7973.473043179992),
 (('3', 'dias'), 7645.367432256628),
 (('boa', 'noite'), 6445.995892016467),
 (('cheiro', 'gosto'), 6187.1150865093),
 (('www', 'youtube'), 6153.505077922475),
 (('corona', 'virus'), 5224.399715203055),
 (('sinto', 'cheiro'), 4572.425125205145),
 (('vitamina', 'c'), 4523.5120698652),
 (('bom', 'dia'), 4473.085106279236),
 (('3', 'comprimidos'), 4406.6050363202885),
 (('lair', 'ribeiro'), 4213.562386985365),
 (('todo'

In [23]:
lr_by_channel = []
for i, channel in enumerate(comments_by_channel):
    lr_scores = score_ngrams(channel, score_metric=lr, freq_filter=10)
    lr_by_channel.append(lr_scores)

df_cc = pd.DataFrame(lr_by_channel, index = channels).T
df_cc.head(10)

,Dr. Alain Dutra,Dr. Alvaro Galvão,Dr. Felipe Ades MD PhD,Dr. Fernando Gomes,Dr. Lair Ribeiro Oficial,Drauzio Varella,Julio Pereira - Neurocirurgião,Lucy Kerr
0,"((dr, alain), 3345.186464110398)","((15, dias), 1433.323840017084)","((gracas, deus), 194.71234851107283)","((dr, fernando), 1233.174621062792)","((dr, lair), 3070.573403026563)","((dr, drauzio), 9946.266396480833)","((olfato, paladar), 9294.89036713873)","((dra, lucy), 4949.857984094305)"
1,"((3, dias), 3138.7450853742057)","((dr, alvaro), 1374.9098465185807)","((falta, ar), 171.6449369048991)","((boa, noite), 673.7437399532348)","((lair, ribeiro), 1912.505909290369)","((falta, ar), 6662.538198743801)","((dor, cabeca), 7693.015303967335)","((youtu, be), 4866.240449175779)"
2,"((deus, abencoe), 2854.907757384603)","((alvaro, galvao), 993.0891522081896)","((boa, noite), 139.84440469962718)","((gomes, pinto), 466.26627184117444)","((covid, 19), 712.1283554510551)","((covid, 19), 4900.174565503226)","((falta, ar), 7523.2471829708165)","((https, www), 4219.331930599844)"
3,"((dra, lucy), 2733.3967374153694)","((dose, unica), 787.3034740336434)","((dor, cabeca), 113.5048027238131)","((fernando, gomes), 395.00970193580923)","((deus, abencoe), 544.6811369363458)","((grupo, risco), 3038.8302928836542)","((gracas, deus), 6941.126882086457)","((lucy, kerr), 4011.237050176198)"
4,"((youtu, be), 2227.722587525147)","((cada, 15), 770.3126459217473)","((covid, 19), 104.93686270125268)","((good, work), 256.20823587178336)","((ministro, saude), 352.3585551070647)","((drauzio, varella), 2902.3104859617974)","((sentir, cheiro), 6016.735248730385)","((https, youtu), 3718.785581598374)"
5,"((3, comprimidos), 2018.8471178616242)","((covid, 19), 694.6179608339949)","((deu, positivo), 99.88573236415296)","((thank, you), 187.15407486528278)","((youtu, be), 300.08992510442374)","((bom, dia), 2835.4465996077615)","((covid, 19), 5192.216671039323)","((www, youtube), 3489.64816096683)"
6,"((dias, seguidos), 2009.700521668765)","((deus, abencoe), 662.6052188862465)","((deus, abencoe), 93.98432367740156)","((deus, abencoe), 164.7882200171475)","((https, youtu), 244.60879405563904)","((99727, 3349), 2580.2148990741007)","((cheiro, gosto), 4323.8918574001855)","((covid, 19), 3371.152878899452)"
7,"((lucy, kerr), 1946.2551243462065)","((1, comprimido), 652.657719475173)","((muita, dor), 89.37578402232545)","((fernando, gomez), 101.80592605426995)","((boa, noite), 216.2972204146699)","((71, 99727), 2550.7628951221805)","((sinto, cheiro), 3072.9998947435583)","((deus, abencoe), 2826.443598911361)"
8,"((covid, 19), 1928.9673912566197)","((3, comprimidos), 515.5022903168635)","((14, dias), 86.88461760270032)","((very, informative), 96.55976887463909)","((https, www), 213.9908984046973)","((todo, mundo), 2438.7841515196255)","((youtu, be), 2687.828348160729)","((15, dias), 2325.550389246373)"
9,"((15, dias), 1918.2792838040386)","((lucy, kerr), 498.9717085069292)","((fiz, teste), 75.3106632397851)","((gomez, pinto), 91.82415028812176)","((boa, tarde), 208.94366947289538)","((watsszzap, 71), 2176.967619061779)","((corona, virus), 2534.241131629531)","((youtube, channel), 1542.9666369548627)"
